# FAISS - Facebook AI Similarity Search

In [2]:
%cd ..

/Users/isham993/Desktop/Programming-Tutorials/2023-Data-Science/Langchain_Advanced_Projects/news-research-tool


In [6]:
pd.set_option('display.max_colwidth', 100)

In [7]:
from utils import *

In [8]:
df = pd.read_csv("./data/sample_text.csv")
df.shape

(8, 2)

In [9]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [10]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [11]:
vectors.shape

(8, 768)

### Step 2 : Build a FAISS Index for vectors

In [13]:
dim = vectors.shape[1]
dim

768

In [14]:
# At this point certain data structure are used
index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [15]:
index.add(vectors)

In [16]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x29458b9c0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [17]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [19]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created

In [21]:
distances, I = index.search(svec, k=3)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [22]:
I.tolist()

[[3, 2]]

In [23]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [24]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [25]:
search_query

'I want to buy a polo t-shirt'

### Put it all together

In [29]:
def return_similar_records(dataframe: pd.DataFrame, search_query: str, top_k: int = 3):
    vec = encoder.encode(search_query)
    svec = np.array(vec).reshape(1,-1)
    distances, I = index.search(svec, k = top_k)
    row_indices = I.tolist()[0]
    return dataframe.loc[row_indices]

In [32]:
return_similar_records(df, "Apple a day keeps doctor away", 2)

,text,category
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
0,Meditation and yoga can improve mental health,Health
